### [Story](https://www.pivotaltracker.com/story/show/187949220)
### [Source](https://medium.com/@prathammodi001/keras-cnn-tutorial-classifying-images-made-easy-fb55cc8892ec)

Create from scratch a CNN apllied to computer vision, to classify cats and dogs from a well known image dataset published in [Kaggle](https://www.kaggle.com/c/dogs-vs-cats/overview).

## Dog Breed Classifier CNN

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="-1" 

import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
print(tf.__version__)
if tf.config.list_logical_devices('GPU'):
    print("Yes,  GPU available")
else:
    print("No GPU available")

### Dataset 

In [ ]:
# Load dataset
pd.set_option('display.max_columns', None) # show all columns
pd.set_option('display.max.rows', None) # showw all rows

labels_csv = pd.read_csv("./dataset/labels.csv")
labels_csv.describe()

In [ ]:
# EDA (exploratory data analysis)
ax = labels_csv["breed"].value_counts().plot.bar(figsize=(20,10))

average_value = labels_csv["breed"].value_counts().mean()
ax.axhline(average_value, color="red", linestyle="--", label="Average")

plt.xlabel("Brred")
plt.ylabel("Count")
plt.title("Breed Distribution")
plt.legend


### Get images and their labels

In [ ]:
from IPython.display import Image

In [ ]:
# Test one image
for image in labels_csv["id"].head(1):
    display(Image(filename="./dataset/train/"+image+".jpg", width=300, height="300"))

In [ ]:
# get al images
filenames = []
for image_id in labels_csv["id"]:
    filenames.append("./dataset/train/"+image_id+".jpg")

filenames

In [ ]:
# check if all filenames of all images have been acquired
if len(os.listdir("./dataset/train")) == len(filenames):
    print("Correct! Filenames match amount of files!")
else:
    print("Error! Filenames do not match amount of files")

### labels

In [ ]:
labels = labels_csv["breed"].to_numpy()

labels

In [ ]:
# check for missing data, lables or images
if len(filenames) == len(labels):
    print("No missing Data!")
else:
    print("Missing data!!!")

In [ ]:
unique_breeds = labels
labels[0] == unique_breeds

boolean_labels = []

for label in labels:
    boolean_labels.append(label == unique_breeds)

## Prepare Train, Valid and Test sets

In [ ]:
X = filenames
y = boolean_labels

NUM_IMAGES= 1000

from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X[:NUM_IMAGES], y[:NUM_IMAGES], test_size=0.2, random_state=42)
len(X_train), len(y_train), len(X_val), len(y_val)

## Images to tensors

1. Take an image as input
2. Use Tensorflow to read the file and save it to a variable `image`
3. Turn the `image` (.jpeg) into Tensors
4. Normalize the `image`
5. Resize the `image` to 224x224
6. Return the processed image

In [ ]:
# test 1
image = plt.imread(filenames[42])
image.shape

In [ ]:
# Test 2. 
tensor = tf.io.read_file(filenames[20])
#tensor

In [ ]:
# Test 3. Normalization: Turn image into numerical Tensor values: 0-255 (RGB)
tensor = tf.image.decode_jpeg(tensor, channels=3)
tensor

In [ ]:
# Test 4. : convert RGB numbers 0-255 into 0-1 for each RGB
tf.image.convert_image_dtype(tensor, tf.float32)

In [ ]:
IMG_SIZE = 224
# Takes an image file path and turns it into a Tensor
def preprocess_image(image_path, img_size=IMG_SIZE):
    # 1. read image
    image = tf.io.read_file(image_path)
    # 2. turn it into numerical Tensor using 3 channels RGB
    image = tf.image.decode_jpeg(image, cahnnels=3)
    # 3. Normalize: convert 0-255 values into 0-1 for each RGB
    image = tf.image.convert_image_dtype(image, tf.float32)
    # 4. Resize to 224x224
    image = tf.image.resize(image, size=[IMG_SIZE,IMG_SIZE])
    # 5 return image
    return image

In [ ]:
# Batch size data: function to return a tuple (image, label)
def get_image_label(image_path, label):
    image = preprocess_image(image_path)
    return image, label   

In [ ]:
BATCH_SIZE = 32
""" Creates batches of data out of image (X) and lable (y) pairs"""
def create_batches(X, y=None, batch_size = BATCH_SIZE, valid_data = False, test_data = False):
    if test_data: # NO labels
        print("Creating Test data Batches...")
        data_whole = tf.data.Dataset.from_tensor_slices((tf.constant(X),))
        data_batch = data_whole.map(preprocess_image).batch(BATCH_SIZE)
        return data_batch

    elif valid_data: 
        print("Creating Valid Data batches...")
        data_whole = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
        data_batch = data_whole.map(get_image_label).batch(BATCH_SIZE)
        return data_batch
    else: # Trainig set
        print("Creating Training Data Batches...")
        data_whole = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
        data_batch = data_whole.shuffle(buffer_size=len(X)).map(get_image_label).batch(BATCH_SIZE)
        return data_batch  

In [ ]:
train_data = create_batches(X_train, y_train)
val_data = create_batches(X_val, y_val, valid_data=True)

### Visualize Data Batches